## Steps for Notebook
1. Getting the datasets (cleveland, census data, acs data) (https://la.arcgis.com/databrowser/index.html), (immigation, emigration) (https://www.census.gov/acs/www/data/data-tables-and-tools/data-profiles/2022/) (**Ethan**)
2. Combine into one dataset via geoenrichment (per block per year) (could be zip_code per year) (**Calvin**)
4. Initial Visualization (hotspots for crime and socioeconomic factors, visualize blocks/zip_codes in Cleveland, show difference in demographics with a hotspot comparing 2010 vs 2020 census) (**Calvin** for hotspot), (**Ethan** for chloropleth maps)
5. Perform correlation analysis with hotspots. How much socioeconomic factors explain the varability in crime rate. (**Ethan**)
6. kNN, Isolation Forest, One-class SVM, Random Forest. Predict the crime rate for a zip-code/block based on socioeconomic factors. Split into a training vs test set (80/20). See which ones have the lowest MSE. (**Ethan** for Random Forest, Isolation Forest) (**Calvin** for kNN and One-class SVM)
8. Comparing the models and detemerining the best one. (**Calvin**)

Finish up to step 4 by Thursday 

## Part 1: Getting Data

## Part 2: Combining Data

## Part 3: Visualizations

## Part 4: Correlation Analysis